# Baseline model

## B0

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../data/train_1M.csv')
questions = pd.read_csv('../data/questions.csv')
train = train.query('content_type_id==0')
train = train.rename(columns={'content_id': 'question_id'})

In [3]:
questions['question_avg_score'] = train.groupby('question_id')['answered_correctly'].mean()

In [4]:
# Only keep questions, rename content_id and merge with questions table.
train = train.merge(questions, on='question_id').sort_values(by=['user_id', 'timestamp'])

In [5]:
# Add features . 
train['user_activity_cumcount']=train.groupby('user_id').cumcount()
#train['question_avg_score'] = train.groupby('question_id')['answered_correctly'].mean()
train['avgscore_cummean'] = train.groupby('user_id').answered_correctly.apply(lambda x: x.shift().expanding().mean())
# Then remove all columns but 'question_avg_score', 'user_avgscore_cummean', 'answered_correctly' (target). Replace inf by 0
b0_train = train[['question_avg_score', 'avgscore_cummean', 'answered_correctly']]
b0_train.avgscore_cummean.replace(np.nan, 0, inplace=True)



/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [9]:
#b0_train.to_csv('b0_model.csv')

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn. metrics import roc_auc_score, accuracy_score

In [18]:
log = LogisticRegression(solver='saga', max_iter=1000)

X = b0_train[['question_avg_score', 'avgscore_cummean']]
y = b0_train.answered_correctly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 1)

In [19]:
#Roc-auc score
log.fit(X_train, y_train)

y_pred = cross_val_predict(log, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.74638283257266


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   25.2s finished


In [20]:
X.shape

(967834, 2)

## B1

In [11]:
X_1 = train[['question_avg_score', 'avgscore_cummean', 'prior_question_elapsed_time', 'timestamp']]
X_1.replace(np.nan, 0, inplace=True)
y_1 = b0_train.answered_correctly

/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [15]:
y_1.shape

(967834,)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.3,random_state = 1)
log = LogisticRegression()
log.fit(X_train, y_train)

/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
y_pred = cross_val_predict(log, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Yohann/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver t

roc-auc: 0.5348689975136507
